In [1]:
import numpy as np
import os
import cv2
import time
import glob
import json
import pandas as pd
import io
import os.path
import matplotlib.pyplot as plt
from PIL import Image

from apiclient.http import MediaFileUpload, MediaIoBaseDownload
from genericpath import exists
import numpy as np
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
# from google.oauth2 import service_account
# from googleapiclient.discovery import build
# from googleapiclient.errors import HttpError as HTTPError
# from oauth2client.service_account import ServiceAccountCredentials
# from google.auth.transport.requests import Request
# from google.oauth2.credentials import Credentials
# from google_auth_oauthlib.flow import InstalledAppFlow
# from googleapiclient.http import MediaFileUpload
# from googleapiclient.http import MediaIoBaseDownload

In [2]:
# # googleと連携する
# files = glob.glob('')
# try:
#     jsonf = files[0]
# except Exception as e:
#     print('Error: Get Google Drive API Credential')
#     raise e


# MIME_TYPE = 'application/vnd.google-apps.document'
# APPLICATION_NAME = 'ipa-google-drive-api-client'
# SCOPES = ['https://www.googleapis.com/auth/drive.file']

# sa_creds = service_account.Credentials.from_service_account_file(jsonf)
# scoped_creds = sa_creds.with_scopes(SCOPES)
# drive_service = build('drive', 'v3', credentials=scoped_creds)

In [3]:
# def importPhoto(path):
#     input_file = path

#     # ローカルファイルの定義
#     media_body = MediaFileUpload(input_file, mimetype=MIME_TYPE, resumable=True)

#     body = {
#         'name': input_file,
#         'mimeType': MIME_TYPE
#     }

#     # 　creat関数でファイルアップロード実行
#     # 同時にOCR読み取りも行う
#     output = drive_service.files().create(
#         body=body,
#         media_body=media_body,
#         # ここで読み込み先言語の指定を行う
#         ocrLanguage="ja",
#         ).execute()
#     outputPhoto(output)

In [4]:
# def outputPhoto(output):
#     request = drive_service.files().export_media(
#         fileId=output['id'],
#         mimeType = "text/plain"
#     )

#     #  出力用テキストファイル名
#     output_path = 'output.txt'

#     fh = io.FileIO(output_path, "wb")
#     downloader = MediaIoBaseDownload(fh, request)
#     done = False
#     while done is False:
#         status, done = downloader.next_chunk()

#     # Google Drive上のファイル削除
#     drive_service.files().delete(fileId=output['id']).execute()

#     # # テキストの取得
#     # with open(output_path) as f:
#     #     mylist = f.read().splitlines()[1:]
#     #     print(mylist)

In [5]:
import time

# 当日レースの騎手名を取得する
# 随時urlは変更

lists = []
for i in range(1, 12):
    time.sleep(1)
    
    # html = 'https://nar.netkeiba.com/race/shutuba.html?race_id=2022461023{}'.format(str(i).zfill(2))
    # html = 'https://nar.netkeiba.com/race/shutuba.html?race_id=2022431026{}'.format(str(i).zfill(2))
    html = 'https://nar.netkeiba.com/race/shutuba.html?race_id=2022551106{}'.format(str(i).zfill(2))

    data = pd.read_html(html)[0]

    hourseNumber = []
    table = []

    for j in range(13):
        table.append(data.columns[j][0])

    data.columns = table

    for j in data['馬番']:
        hourseNumber.append(j)

    lists.append(hourseNumber)

In [6]:
# バッチファイルを動かす関数
def mainRace(raceNum):
    print('バッチ開始')
    os.system('/saveMovie.bat')
    print('バッチ終了')

    if(os.path.exists('/horse.ts')):
        print('動画変換開始')
        os.system('/changeFile.bat')
        print('動画変換終了')
        taskRace(raceNum)

In [7]:
def ocr(model, output_path):
    # 入力画像のパラメータ
    img_width = 32 # 入力画像の幅
    img_height = 32 # 入力画像の高さ
    img_ch = 3 # 3ch画像（RGB）

    # 入力データ数
    num_data = 1

    # 画像の読み込み（32×32にリサイズ）
    # 正規化, 4次元配列に変換（モデルの入力が4次元なので合わせる）
    img = load_img(output_path, target_size=(img_width, img_height))
    img = img_to_array(img) 
    img = img.astype('float32')/255.0
    img = np.array([img])

    # 分類機に入力データを与えて予測（出力：各クラスの予想確率）
    y_pred = model.predict(img)

    # 最も確率の高い要素番号
    number_pred = np.argmax(y_pred) 
    if 0.8 < y_pred[0][number_pred]:
        kd = 9
        # print('number_pred : ' + str(number_pred))
    else:
        number_pred = 99
        # print('number_pred : ' + str(number_pred))

    return number_pred

In [8]:
# 映像から馬の画像を抽出する
def taskRace(raceNum):
    fileName = "horse01.mp4"
    movieImg = "No_1.jpg"
    output_path = "output.png"
    # 保存したモデル構造の読み込み
    model = model_from_json(open("model_5.json", 'r').read())
    # 保存した学習済みの重みを読み込み
    model.load_weights("weight_5.hdf5")

    hourseNumber = lists[raceNum]

    print('パドック馬保存開始')
    videoCapture = cv2.VideoCapture(fileName)
    totalFrames = videoCapture.get(cv2.CAP_PROP_FRAME_COUNT)
    print('合計フレーム数 : ' + str(totalFrames))

    frameIndex = 0
    count = 1
    num = 0
    while(frameIndex < totalFrames):
        if(frameIndex >= 0 & frameIndex < totalFrames):
                videoCapture.set(cv2.CAP_PROP_POS_FRAMES, frameIndex)
                ret, image = videoCapture.read()
                plt.imsave(movieImg,image)

                #保存した画像をグレー化してサイズ加工
                imgHorse = Image.open(movieImg).convert('L') 
                img = imgHorse.crop((190, 540, 360, 640)) # (left, upper, right, lower)
                img.save(output_path)

                # 自作OCR適用
                ocrNum = ocr(model, output_path)
                
                try:
                    for hNum in hourseNumber:
                        if str(hNum) == str(ocrNum):
                            savepath = "/race{}/".format(str(raceNum + 1))
                            savepath = savepath + str(hNum)
                            # フォルダがなかったら作成
                            if(True != (os.path.isdir(savepath))):
                                os.mkdir(savepath)
                            imgHorse.save(savepath + "/horse_{}.png".format(num)) 
                            num += 1
                            frameIndex += 30
                            break
                except:
                    os.remove(movieImg)
                    count += 1
                    frameIndex += 30
                    continue
                os.remove(movieImg)
                count += 1
                frameIndex += 60
        else:
            break
        

In [9]:
import schedule
import time


# 実行job関数
def job(x):
    
    mainRace(x)

    if(os.path.exists('/horse01.mp4')):
        os.remove('/horse.ts') 
        os.remove("/horse01.mp4")
        print("終了")


while True:
    schedule.run_pending()
    schedule.every().day.at("15:03").do(job, x=0)
    schedule.every().day.at("15:32").do(job, x=1)
    schedule.every().day.at("16:07").do(job, x=2)
    schedule.every().day.at("16:37").do(job, x=3)
    schedule.every().day.at("17:07").do(job, x=4)
    schedule.every().day.at("17:37").do(job, x=5)
    schedule.every().day.at("18:12").do(job, x=6)
    schedule.every().day.at("18:42").do(job, x=7)
    schedule.every().day.at("19:12").do(job, x=8)
    schedule.every().day.at("19:42").do(job, x=9)
    schedule.every().day.at("20:18").do(job, x=10)
    time.sleep(57)

バッチ開始
バッチ終了
動画変換開始
動画変換終了
パドック馬保存開始
合計フレーム数 : 27000.0
1/1 [==============================] - 0s 16ms/step
終了
バッチ開始
